# 7B based - 8x80GB Vram benchmark

The following is for benchmarking 7B training on 8 x 80GB vram based nvidia cards.
With the following settings.
- 16k data pack size
- 4k training size
- microbatch 10

The following are expected per GPU numbers

| GPU Model | Deepspeed 2 | Deepspeed 3 |
|-----------|-------------|-------------|
| H100 SXM  | 7 kT/s      | -           |
| H100 PCIe | 4.2 kT/s    | -           |
| A100 SXM  | 3 kT/s      | 2.6 kT/s    |
| A100 PCIe | 2.6 kT/s    | 2.3 kT/s    |
| H800 SXM* | 7 kT/s      | -           |

H800 is the "china safe export" edition of H100, with its numbers coming from the RWKV-LM repo, with different settings (not infctx repo). Left here for reference.

Blanks means we did'nt run them (yet?).

In [2]:
GPU_DEVICES="auto"
ENABLE_WANDB=True
WANDB_PREFIX="infctx-v5-selective-loss"
DEEPSPEED_STRAT="deepspeed_stage_2"

print("ENABLE_WANDB:", ENABLE_WANDB)
print("GPU_DEVICES:", GPU_DEVICES)

if ENABLE_WANDB:
    WANDB_MODE="online"
else:
    WANDB_MODE="disabled"

# The model sizing
MODEL_NAME="RWKV-5-World-7B-v2-OnlyForTest_72%_trained-20231204-ctx4096.pth"

# Computing the notebook, and various paths
import os
NOTEBOOK_DIR=os.path.dirname(os.path.abspath("__file__"))
PROJECT_DIR=os.path.abspath(os.path.join(NOTEBOOK_DIR, "../../"))
TRAINER_DIR=os.path.abspath(os.path.join(PROJECT_DIR, "./RWKV-v5/"))

print("NOTEBOOK_DIR:", NOTEBOOK_DIR)
print("TRAINER_DIR:", TRAINER_DIR)
print("PROJECT_DIR:", PROJECT_DIR)

ENABLE_WANDB: True
GPU_DEVICES: auto
NOTEBOOK_DIR: /home/recursal/RWKV-infctx-trainer/notebook/trainer-v5-validation
TRAINER_DIR: /home/recursal/RWKV-infctx-trainer/RWKV-v5
PROJECT_DIR: /home/recursal/RWKV-infctx-trainer


In [5]:
# Lets wget the model files
!mkdir -p "{PROJECT_DIR}/model"
!cd "{PROJECT_DIR}/model" && \
    wget -O "{MODEL_NAME}" -nc "https://huggingface.co/BlinkDL/temp/resolve/2d905a2a30c778086a048e4f65ca75d9f7f9849d/RWKV-5-World-7B-v2-OnlyForTest_72%25_trained-20231204-ctx4096.pth?download=true"

--2024-01-20 05:49:01--  https://huggingface.co/BlinkDL/temp/resolve/2d905a2a30c778086a048e4f65ca75d9f7f9849d/RWKV-5-World-7B-v2-OnlyForTest_72%25_trained-20231204-ctx4096.pth?download=true
Resolving huggingface.co (huggingface.co)... 13.33.33.20, 13.33.33.110, 13.33.33.55, ...
Connecting to huggingface.co (huggingface.co)|13.33.33.20|:443... connected.
HTTP request sent, awaiting response... 

302 Found
Location: https://cdn-lfs.huggingface.co/repos/20/96/209600910186b29a71a230a87a5a555fd33bc07c57dabd6f6f9fd03523c5326f/1bc5c11ab576f130752f004698fd45e278094c671a4f6c067aeeeee9ebe74a48?response-content-disposition=attachment%3B+filename*%3DUTF-8%27%27RWKV-5-World-7B-v2-OnlyForTest_72%2525_trained-20231204-ctx4096.pth%3B+filename%3D%22RWKV-5-World-7B-v2-OnlyForTest_72%25_trained-20231204-ctx4096.pth%22%3B&Expires=1705960141&Policy=eyJTdGF0ZW1lbnQiOlt7IkNvbmRpdGlvbiI6eyJEYXRlTGVzc1RoYW4iOnsiQVdTOkVwb2NoVGltZSI6MTcwNTk2MDE0MX19LCJSZXNvdXJjZSI6Imh0dHBzOi8vY2RuLWxmcy5odWdnaW5nZmFjZS5jby9yZXBvcy8yMC85Ni8yMDk2MDA5MTAxODZiMjlhNzFhMjMwYTg3YTVhNTU1ZmQzM2JjMDdjNTdkYWJkNmY2ZjlmZDAzNTIzYzUzMjZmLzFiYzVjMTFhYjU3NmYxMzA3NTJmMDA0Njk4ZmQ0NWUyNzgwOTRjNjcxYTRmNmMwNjdhZWVlZWU5ZWJlNzRhNDg%7EcmVzcG9uc2UtY29udGVudC1kaXNwb3NpdGlvbj0qIn1dfQ__&Signature=IxiX31-ebesq21xv4qkvpg6mAPXpnSh1MNiaba%7E%7EXEuvqBUd8jCh1SAwRKOnTUbJrQRye6AKOZg1AvrxYfHDP7dj4%7EUs0KCF-W0H4dFEe1MZv4mpE9xKG7FNaqStUtOh-XMSbpj7s30eWnhiUJT

In [3]:
# Lets preload the requried dataset 
!cd "{TRAINER_DIR}" && \
    python3 preload_datapath.py "{NOTEBOOK_DIR}/config/enwiki_100k-world-16k-packing.yaml"

Map (num_proc=160): 100%|███| 1000000/1000000 [00:10<00:00, 95906.26 examples/s]
Filter (num_proc=160): 100%|█| 1000000/1000000 [00:05<00:00, 179927.28 examples/
Map (num_proc=160): 100%|█████| 120800/120800 [00:05<00:00, 21204.90 examples/s]
Saving the dataset (4/4 shards): 100%|█| 18147/18147 [00:04<00:00, 4124.48 examp
Saving the dataset (1/1 shards): 100%|█| 13423/13423 [00:00<00:00, 25885.58 exam


# Actual training run

In [ ]:

!cd "{TRAINER_DIR}" && \
    export WANDB_MODE="{WANDB_MODE}" && \
    python3 lightning_trainer.py fit \
        -c "{NOTEBOOK_DIR}/config/enwiki_100k-world-16k-packing.yaml" \
        --model.load_model="../model/{MODEL_NAME}" \
        --data.skip_datapath_setup=True \
        --trainer.callbacks.init_args.dirpath="../checkpoint/v5-7b-benchmark/baseline/" \
        --trainer.logger.init_args.name="{WANDB_PREFIX} - 7B - Baseline (packsize=16k, {DEEPSPEED_STRAT})" \
        --trainer.strategy="{DEEPSPEED_STRAT}" \
        --trainer.target_batch_size=640 \
        --trainer.microbatch_size=10 \
        --model.ctx_len=4096 \
        --trainer.devices="{GPU_DEVICES}"